In [5]:
# define directory where to save the logs
import os
root_logdir = os.path.join(os.curdir, "my_logs")

In [6]:
# each log will be saved in a different subdirectory.
# function to produce the name of the subdirectories
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

In [7]:
run_logdir = get_run_logdir()  # e.g., './my_logs/run_2023_02_21-10_24_08'

In [4]:
from tensorflow import keras

In [5]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [6]:
# download dataset
housing = fetch_california_housing()

In [7]:
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

In [8]:
# scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [9]:
# build model
input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden1 = keras.layers.Dense(30, activation="relu")(input_)  # connects the input with the first hidden layer
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)  # connects both hidden layers
concat = keras.layers.Concatenate()([input_, hidden2])  # connects the input with the second hidden layer into a new concatenated layer
output = keras.layers.Dense(1)(concat)  # connects the concatenated layer to the output
model = keras.Model(inputs=[input_], outputs=[output])

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-02-21 10:27:39.282720: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-21 10:27:39.282876: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [10]:
# compile
model.compile(loss='mean_squared_error', optimizer=keras.optimizers.SGD(lr=0.001))

/Users/sebastian/Documents/GitHub/study/study_env/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [11]:
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir, )

history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid),
                    callbacks=[tensorboard_cb])

Epoch 1/30


2023-02-21 10:27:52.866503: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-21 10:27:52.999571: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


363/363 [==============================] - ETA: 0s - loss: 1.6323

2023-02-21 10:27:55.078630: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


363/363 [==============================] - 3s 6ms/step - loss: 1.6323 - val_loss: 1.0186
Epoch 2/30
363/363 [==============================] - 2s 6ms/step - loss: 0.6798 - val_loss: 0.6885
Epoch 3/30
363/363 [==============================] - 2s 6ms/step - loss: 0.6067 - val_loss: 0.6152
Epoch 4/30
363/363 [==============================] - 2s 6ms/step - loss: 0.5638 - val_loss: 0.6793
Epoch 5/30
363/363 [==============================] - 2s 6ms/step - loss: 0.5320 - val_loss: 0.6218
Epoch 6/30
363/363 [==============================] - 2s 6ms/step - loss: 0.5074 - val_loss: 0.5971
Epoch 7/30
363/363 [==============================] - 2s 6ms/step - loss: 0.4876 - val_loss: 0.5761
Epoch 8/30
363/363 [==============================] - 2s 6ms/step - loss: 0.4719 - val_loss: 0.6594
Epoch 9/30
363/363 [==============================] - 2s 6ms/step - loss: 0.4594 - val_loss: 0.6456
Epoch 10/30
363/363 [==============================] - 2s 6ms/step - loss: 0.4485 - val_loss: 0.7056
Epoch 11/3

In [1]:
# start tensorboard server
# tensorboard --logdir=./my_logs --port 6006


# Custom statistics

In [3]:
import tensorflow as tf
import numpy as np

In [14]:
test_logdir = get_run_logdir()
writer = tf.summary.create_file_writer(test_logdir)
with writer.as_default():
    for step in range(1, 1000 + 1):
        tf.summary.scalar("my_scalar", np.sin(step / 10), step=step)
        # histograms (more specifically tf.zeros_like) seems to be an issue with the version of tensorflow-macos
        # data = (np.random.randn(100) + 2) * step / 100  # some random data
        # tf.summary.histogram("my_hist", tf.convert_to_tensor(data), buckets=50, step=step)
        images = np.random.rand(2, 32, 32, 3)  # random 32x32 RGB images
        tf.summary.image("my_images", images * step / 1000, step=step)
        texts = [f"The step is {step}", f"Its square is {step**2}"]
        tf.summary.text("my_text", texts, step=step)
        sine_wave = tf.math.sin(tf.range(12000) / 48000 * 2 * np.pi * step)
        audio = tf.reshape(tf.cast(sine_wave, tf.float32), [1, -1, 1])
        tf.summary.audio("my_audio", audio, sample_rate=48000, step=step)
        
